# File QA RAG Chatbot App with ChatGPT, LangChain and Streamlit

Here we will implement an advanced RAG System with ChatGPT, LangChain and Streamlit to build a File QA UI-based chatbot with the following features:

- PDF Document Upload and Indexing
- RAG System for query analysis and response
- Result streaming capabilities (Real-time output)
- Show document sources of the answer from RAG system

## Install App and LLM dependencies

In [6]:
%pip install langchain==0.1.12
%pip install openai==0.27.4
# %pip install langchain-openai==0.0.8
# %pip install langchain-community==0.0.29
# %pip install streamlit==1.32.2
# %pip install PyMuPDF==1.24.0
# %pip install chromadb==0.4.24
# %pip install pyngrok==7.1.5

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
pip install --upgrade openai

   ---------------------------------------- 567.4/567.4 kB 6.5 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 0.27.4
    Uninstalling openai-0.27.4:
      Successfully uninstalled openai-0.27.4
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Load OpenAI API Credentials

Here we load it from a file so we don't explore the credentials on the internet by mistake

In [21]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [22]:
import yaml

with open('openai_key.yml', 'r') as file:
    api_creds = yaml.safe_load(file)

In [23]:
api_creds.keys()

dict_keys(['openai_key'])

In [24]:
import os

os.environ['OPENAI_API_KEY'] = api_creds['openai_key']

In [32]:
#(api_creds['openai_key'])

## Write the app code here and store it in a py file

In [ ]:
#%%writefile olgafn.py
# the following line is a magic command
# that will write all the code below it to the python file app.py
# we will then deploy this app.py file on the cloud server where colab is running
# if you have your own server you can just write the code in app.py and deploy it directly

import locale
locale.getpreferredencoding = lambda: "UTF-8"

import yaml
with open('openai_key.yml', 'r') as file:
    api_creds = yaml.safe_load(file)
    
api_creds.keys()

import os

os.environ['OPENAI_API_KEY'] = api_creds['openai_key']

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_message_histories import StreamlitChatMessageHistory
from langchain_core.callbacks.base import BaseCallbackHandler
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores.chroma import Chroma
from operator import itemgetter
import streamlit as st
import tempfile
import os
import pandas as pd

# Customize initial app landing page
st.set_page_config(page_title="File QA Chatbot", page_icon="🤖")
st.title("Welcome to OLGA QA RAG Chatbot 🤖")
st.image("logo.jpg", width=400) #add logo. Adjust width as necessary

@st.cache_resource(ttl="1h")
# Takes uploaded PDFs, creates document chunks, computes embeddings
# Stores document chunks and embeddings in a Vector DB
# Returns a retriever which can look up the Vector DB
# to return documents based on user input
# Stores this in the cache
def configure_retriever(uploaded_files):
  # Read documents
  docs = []
  temp_dir = tempfile.TemporaryDirectory()
  for file in uploaded_files:
    temp_filepath = os.path.join(temp_dir.name, file.name)
    with open(temp_filepath, "wb") as f:
      f.write(file.getvalue())
    loader = PyMuPDFLoader(temp_filepath)
    docs.extend(loader.load())

  # Split into documents chunks
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500,
                                                 chunk_overlap=200)
  doc_chunks = text_splitter.split_documents(docs)

  # Create document embeddings and store in Vector DB
  embeddings_model = OpenAIEmbeddings(openai_api_base="https://api.openai.com/v1")
  #embeddings_model = OpenAIEmbeddings()
  vectordb = Chroma.from_documents(doc_chunks, embeddings_model)

  # Define retriever object
  retriever = vectordb.as_retriever()
  return retriever

# Manages live updates to a Streamlit app's display by appending new text tokens
# to an existing text stream and rendering the updated text in Markdown
class StreamHandler(BaseCallbackHandler):
  def __init__(self, container, initial_text=""):
    self.container = container
    self.text = initial_text

  def on_llm_new_token(self, token: str, **kwargs) -> None:
    self.text += token
    self.container.markdown(self.text)

# Creates UI element to accept PDF uploads
uploaded_files = st.sidebar.file_uploader(
  label="Upload PDF files here", type=["pdf"],
  accept_multiple_files=True
)
if not uploaded_files:
  st.info("Please upload PDF documents to continue.")
  st.stop()

# Create retriever object based on uploaded PDFs
retriever = configure_retriever(uploaded_files)

# Load a connection to ChatGPT LLM
chatgpt = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0,
                     streaming=True)

# Create a prompt template for QA RAG System
qa_template = """
              Use only the following pieces of context to answer the question at the end.
              If you don't know the answer, just say that you don't know,
              don't try to make up an answer. Keep the answer as concise as possible.

              {context}

              Question: {question}
              """
qa_prompt = ChatPromptTemplate.from_template(qa_template)

# This function formats retrieved documents before sending to LLM
def format_docs(docs):
  return "\n\n".join([d.page_content for d in docs])

# Create a QA RAG System Chain
qa_rag_chain = (
  {
    "context": itemgetter("question") # based on the user question get context docs
      |
    retriever
      |
    format_docs,
    "question": itemgetter("question") # user question
  }
    |
  qa_prompt # prompt with above user question and context
    |
  chatgpt # above prompt is sent to the LLM for response
)

# Store conversation history in Streamlit session state
streamlit_msg_history = StreamlitChatMessageHistory(key="langchain_messages")

# Shows the first message when app starts
if len(streamlit_msg_history.messages) == 0:
  streamlit_msg_history.add_ai_message("What is your name and how can I help you?")

# Render current messages from StreamlitChatMessageHistory
for msg in streamlit_msg_history.messages:
  st.chat_message(msg.type).write(msg.content)

# Callback handler which does some post-processing on the LLM response
# Used to post the top 3 document sources used by the LLM in RAG response
class PostMessageHandler(BaseCallbackHandler):
  def __init__(self, msg: st.write):
    BaseCallbackHandler.__init__(self)
    self.msg = msg
    self.sources = []

  def on_retriever_end(self, documents, *, run_id, parent_run_id, **kwargs):
    source_ids = []
    for d in documents: # retrieved documents from retriever based on user query
      metadata = {
        "source": d.metadata["source"],
        "page": d.metadata["page"],
        "content": d.page_content[:100]
      }
      idx = (metadata["source"], metadata["page"])
      if idx not in source_ids: # store unique source documents
        source_ids.append(idx)
        self.sources.append(metadata)

  def on_llm_end(self, response, *, run_id, parent_run_id, **kwargs):
    if len(self.sources):
      st.markdown("__Sources:__ "+"\n")
      st.dataframe(data=pd.DataFrame(self.sources[:3]),
                    width=1000) # Top 3 sources


# If user inputs a new prompt, display it and show the response
if user_prompt := st.chat_input():
  st.chat_message("human").write(user_prompt)
  # This is where response from the LLM is shown
  with st.chat_message("ai"):
    # Initializing an empty data stream
    stream_handler = StreamHandler(st.empty())
    # UI element to write RAG sources after LLM response
    sources_container = st.write("")
    pm_handler = PostMessageHandler(sources_container)
    config = {"callbacks": [stream_handler, pm_handler]}
    # Get LLM response
    response = qa_rag_chain.invoke({"question": user_prompt},
                                    config)

Overwriting olgafn.py


## Start the app

In [ ]:
#!streamlit run app.py --server.port=8989 &>./logs.txt &
streamlit run olgafn.py server.port=8989 *>./logs.txt 2>&1

# run in the background without opening a new window. Grab the url from the .txt file
#Start-Process -FilePath "chainlit" -ArgumentList "run olga.py --port=8989 --watch" -RedirectStandardOutput "logs.txt" -RedirectStandardError "logs.txt" -NoNewWindow


In [ ]:
# from pyngrok import ngrok
# import yaml

# # Terminate open tunnels if exist
# ngrok.kill()

# # Setting the authtoken
# # Get your authtoken from `ngrok_credentials.yml` file
# with open('ngrok_credentials.yml', 'r') as file:
#     NGROK_AUTH_TOKEN = yaml.safe_load(file)
# ngrok.set_auth_token(NGROK_AUTH_TOKEN['ngrok_key'])

# # Open an HTTPs tunnel on port XXXX which you get from your `logs.txt` file
# ngrok_tunnel = ngrok.connect(8989)
# print("Streamlit App:", ngrok_tunnel.public_url)

Streamlit App: https://70cd-35-185-85-25.ngrok-free.app


## Remove running app processes

In [31]:
#ngrok.kill()

In [ ]:
!ps -ef | grep streamlit

root        1110       1  1 05:29 ?        00:00:20 /usr/bin/python3 /usr/local/bin/streamlit run ap
root        9062     306  0 06:02 ?        00:00:00 /bin/bash -c ps -ef | grep streamlit
root        9064    9062  0 06:02 ?        00:00:00 grep streamlit


In [ ]:
!sudo kill -9 1110

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

import yaml
with open('openai_key.yml', 'r') as file:
    api_creds = yaml.safe_load(file)
    
api_creds.keys()

import os

os.environ['OPENAI_API_KEY'] = api_creds['openai_key']

In [ ]:
#(api_creds['openai_key'])

This confirms my API key is valid and making calls to this model.

In [ ]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
import openai
openai.api_key = 'openai_key'

try:
    response = OpenAIEmbeddings(
        input="Test text",
        model="text-embedding-ada-002"
    )
    print("Access confirmed. Embedding response:", response)
except Exception as e:
    print("Error:", e)

Access confirmed. Embedding response: client=<openai.resources.embeddings.Embeddings object at 0x000001C170C27880> async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x000001C170CCDFD0> model='text-embedding-ada-002' dimensions=None deployment='text-embedding-ada-002' openai_api_version='' openai_api_base=None openai_api_type='' openai_proxy='' embedding_ctx_length=8191 openai_api_key=SecretStr('**********') openai_organization=None allowed_special=set() disallowed_special='all' chunk_size=1000 max_retries=2 request_timeout=None headers=None tiktoken_enabled=True tiktoken_model_name=None show_progress_bar=False model_kwargs={'input': 'Test text'} skip_empty=False default_headers=None default_query=None retry_min_seconds=4 retry_max_seconds=20 http_client=None


In [8]:
import streamlit
print(streamlit.__version__)

1.43.2
